In [107]:
period=-59
time_interval='15m'
tickers = ['EURUSD=X', 'GBPUSD=X', 'USDCHF=X', 'USDJPY=X']
forex_tickers = ['EURUSD', 'GBPUSD', 'USDCHF', 'USDJPY']


In [108]:
# Load data
%run forex_data_loader.ipynb

yfinance version: 0.2.50
Start load FOREX data interval with: 15m, from: 2024-10-03 22:14:27.924770


[*********************100%***********************]  4 of 4 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3888 entries, 2024-10-03 22:00:00 to 2024-11-29 22:15:00
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   (USDJPY=X, Open)       3870 non-null   float64
 1   (USDJPY=X, High)       3870 non-null   float64
 2   (USDJPY=X, Low)        3870 non-null   float64
 3   (USDJPY=X, Close)      3870 non-null   float64
 4   (USDJPY=X, Adj Close)  3870 non-null   float64
 5   (USDJPY=X, Volume)     3870 non-null   float64
 6   (EURUSD=X, Open)       3885 non-null   float64
 7   (EURUSD=X, High)       3885 non-null   float64
 8   (EURUSD=X, Low)        3885 non-null   float64
 9   (EURUSD=X, Close)      3885 non-null   float64
 10  (EURUSD=X, Adj Close)  3885 non-null   float64
 11  (EURUSD=X, Volume)     3885 non-null   float64
 12  (USDCHF=X, Open)       3874 non-null   float64
 13  (USDCHF=X, High)       3874 non-null   float64
 14  (USDCHF=X, Low)     

In [109]:
import talib as ta
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import backtesting
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

In [110]:
%run file_loader.ipynb
%run useful_functions.ipynb

In [117]:
dir = "forex_data"
forex_dict = {}
for name in forex_tickers:
    f = os.path.join(dir, name+'.csv')
    if os.path.isfile(f):
        df = loading_v3(f)

        df["EMA"]=ta.EMA(df.Close, timeperiod=200)
        df['MACD'], df['MACD_signal'], df['MACD_hist'] = ta.MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)
        df["ATR"] = ta.ATR(low = df['Low'], close = df['Close'], high = df['High'], timeperiod=7)


        df = df.sort_values('Date').reset_index(drop=True)
        df = df.set_index('Date')

        forex_dict[name] = df


In [118]:
forex_dict['EURUSD'].describe()

,Open,High,Low,Close,Volume,chg,vol_chg,EMA,MACD,MACD_signal,MACD_hist,ATR
count,3885.000000,3885.000000,3885.000000,3885.000000,3885.0,3885.000000,3885.0,3686.000000,3852.000000,3852.000000,3.852000e+03,3878.000000
mean,1.075356,1.075608,1.075100,1.075346,0.0,-0.001120,0.0,1.075226,-0.000084,-0.000085,7.509185e-07,0.000541
std,0.016587,0.016528,0.016643,0.016585,0.0,0.048783,0.0,0.015873,0.000571,0.000536,1.751020e-04,0.000308
min,1.034233,1.040150,1.034233,1.037667,0.0,-0.760000,0.0,1.047420,-0.004492,-0.003813,-1.156192e-03,0.000167
25%,1.057530,1.057865,1.057194,1.057418,0.0,-0.020000,0.0,1.057318,-0.000334,-0.000328,-7.857518e-05,0.000328
50%,1.080731,1.080964,1.080497,1.080731,0.0,0.000000,0.0,1.081113,-0.000050,-0.000059,5.419867e-06,0.000454
75%,1.088613,1.088732,1.088376,1.088495,0.0,0.020000,0.0,1.086891,0.000213,0.000193,8.676582e-05,0.000670
max,1.104118,1.104240,1.103997,1.104240,0.0,0.580000,0.0,1.099322,0.001828,0.001644,8.870937e-04,0.002725


In [ ]:
for name, df in forex_dict.items():
    print('EMASignal for ' + name)
    df = df[-20000:-1]
    df.reset_index(inplace=True)

    df = add_ema_signal(df)
    
    forex_dict[name] = df


EMASignal for EURUSD


100%|██████████| 3884/3884 [00:05<00:00, 670.77it/s]
/tmp/ipykernel_5493/106517577.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



EMASignal for GBPUSD


100%|██████████| 3884/3884 [00:05<00:00, 730.23it/s]
/tmp/ipykernel_5493/106517577.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



EMASignal for USDCHF


100%|██████████| 3873/3873 [00:05<00:00, 660.62it/s]
/tmp/ipykernel_5493/106517577.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



EMASignal for USDJPY


100%|██████████| 3869/3869 [00:05<00:00, 718.30it/s]
/tmp/ipykernel_5493/106517577.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [120]:
forex_dict['EURUSD']

,Date,Open,High,Low,Close,Volume,chg,vol_chg,EMA,MACD,MACD_signal,MACD_hist,ATR,EMASignal
0,2024-10-03 22:00:00,1.103387,1.103631,1.103387,1.103509,0.0,0.00,0.0,NaN,NaN,NaN,NaN,NaN,0
1,2024-10-03 22:15:00,1.103509,1.103631,1.103387,1.103509,0.0,0.00,0.0,NaN,NaN,NaN,NaN,NaN,0
2,2024-10-03 22:30:00,1.103509,1.103753,1.103387,1.103509,0.0,0.00,0.0,NaN,NaN,NaN,NaN,NaN,0
3,2024-10-03 22:45:00,1.103631,1.103631,1.103387,1.103631,0.0,0.01,0.0,NaN,NaN,NaN,NaN,NaN,0
4,2024-10-03 23:00:00,1.103509,1.103753,1.103509,1.103753,0.0,0.01,0.0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3879,2024-11-29 21:00:00,1.058537,1.058537,1.058201,1.058201,0.0,-0.03,0.0,1.055690,0.000642,0.000514,0.000128,0.000429,2
3880,2024-11-29 21:15:00,1.058313,1.058313,1.058089,1.058201,0.0,0.00,0.0,1.055715,0.000614,0.000534,0.000080,0.000400,2
3881,2024-11-29 21:30:00,1.058201,1.058425,1.058201,1.058201,0.0,0.00,0.0,1.055740,0.000585,0.000544,0.000041,0.000375,2
3882,2024-11-29 21:45:00,1.058537,1.058537,1.058089,1.058089,0.0,-0.01,0.0,1.055763,0.000547,0.000545,0.000002,0.000385,2


In [ ]:
for name, df in forex_dict.items():
    print('Signals for ' + name)
    df = add_signal_by_ema_macd(df)
    forex_dict[name] = df

Signals for EURUSD


100%|██████████| 3884/3884 [00:15<00:00, 253.08it/s]


Signals for GBPUSD


100%|██████████| 3884/3884 [00:14<00:00, 264.85it/s]


Signals for USDCHF


100%|██████████| 3873/3873 [00:14<00:00, 263.78it/s]


Signals for USDJPY


100%|██████████| 3869/3869 [00:15<00:00, 257.36it/s]


In [122]:
sum([forex_dict[key]['Signal'].value_counts() for key in forex_dict], start=0)

Signal
0    15236
1      152
2      122
Name: count, dtype: int64

In [123]:
for name, df in forex_dict.items():
    print("Validate Signal " + name)
    print(df[df.Signal != 0].head(20))

Validate Signal EURUSD
                    Date      Open      High       Low     Close  Volume  \
209  2024-10-08 03:45:00  1.097936  1.098418  1.097936  1.098418     0.0   
221  2024-10-08 06:45:00  1.098539  1.098660  1.098418  1.098660     0.0   
290  2024-10-09 01:00:00  1.097815  1.097936  1.097695  1.097815     0.0   
318  2024-10-09 08:00:00  1.096732  1.096732  1.095530  1.095530     0.0   
338  2024-10-09 13:00:00  1.095890  1.096131  1.095890  1.095890     0.0   
405  2024-10-10 05:45:00  1.094331  1.094451  1.094092  1.094451     0.0   
444  2024-10-10 15:30:00  1.092657  1.092657  1.091822  1.092061     0.0   
480  2024-10-11 00:30:00  1.093613  1.093733  1.093613  1.093733     0.0   
502  2024-10-11 06:00:00  1.093853  1.093972  1.093733  1.093853     0.0   
636  2024-10-14 17:00:00  1.090988  1.091107  1.090631  1.090631     0.0   
672  2024-10-15 02:00:00  1.090988  1.091107  1.090869  1.090988     0.0   
714  2024-10-15 12:30:00  1.090394  1.090988  1.090394  1.090988 

In [124]:
for name, df in forex_dict.items():
    print("Position point " + name)

    df = add_pointpos_column(df, 'Signal', 1e-3) # 1e-3 = 0.001, 1e-4=0.0001
    
    forex_dict[name] = df

Position point EURUSD
Params: 0.001
Position point GBPUSD
Params: 0.001
Position point USDCHF
Params: 0.001
Position point USDJPY
Params: 0.001


In [125]:
st=1100
for name, df in forex_dict.items():
    dfpl = df[st:st+350]

    # Create a plot with 2 rows
    fig = make_subplots(rows=2, cols=1, column_titles=[name])

    # Add candlestick plot on the first row
    fig.add_trace(go.Candlestick(x=dfpl.index,
                                open=dfpl['Open'],
                                high=dfpl['High'],
                                low=dfpl['Low'],
                                close=dfpl['Close']),
                row=1, col=1)

    fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                    marker=dict(size=5, color="MediumPurple"),
                    name="entry"),
                row=1, col=1)
    
    fig.update_layout(width=800, height=400, sliders=[])
    fig.show()

In [126]:
%run backtest_strategy.ipynb

In [ ]:
results = dict()

for name, df in forex_dict.items():
    bt = Backtest(df, MacdStrategy, cash=100000, margin=1/30, commission=0.0001)
    stats, heatmap = bt.optimize(slcoef=[i/10 for i in range(10, 26)],
                    TPcoef=[i/10 for i in range(10, 26)],
                    maximize='Return [%]', max_tries=300,
                    random_state=0,
                    return_heatmap=True)
    results[name] = [stats, heatmap]

/tmp/ipykernel_5493/3930247105.py:4: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Backtest.optimize:   0%|          | 0/8 [00:00<?, ?it/s]

/tmp/ipykernel_5493/3930247105.py:4: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Backtest.optimize:   0%|          | 0/8 [00:00<?, ?it/s]

/tmp/ipykernel_5493/3930247105.py:4: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Backtest.optimize:   0%|          | 0/8 [00:00<?, ?it/s]

/tmp/ipykernel_5493/3930247105.py:4: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Backtest.optimize:   0%|          | 0/8 [00:00<?, ?it/s]

In [134]:
for name, result_list in results.items():
    print("=== Stats of {} ===".format(name))
    print(result_list[0])


=== Stats of EURUSD ===
Start                                     0.0
End                                    3883.0
Duration                               3883.0
Exposure Time [%]                    7.415036
Equity Final [$]                  9973.185345
Equity Peak [$]                       10000.0
Return [%]                          -0.268147
Buy & Hold Return [%]               -4.115965
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.292903
Avg. Drawdown [%]                   -0.292903
Max. Drawdown Duration                 3674.0
Avg. Drawdown Duration                 3674.0
# Trades                                 72.0
Win Rate [%]                             37.5
Best Trade [%]                        0.22706
Worst Trade [%]                     -0.567654
Avg. Trade

In [140]:
for name, result_list in results.items():
    stats = result_list[0]
    print("=== Return of {}, {} ===".format(name, stats["Return [%]"]))

=== Return of EURUSD, -0.002681465456821024 ===
=== Return of GBPUSD, 0.0012804774987278506 ===
=== Return of USDCHF, -0.0013314405111479572 ===
=== Return of USDJPY, -0.23968635542477712 ===


In [141]:
for name, result_list in results.items():
    stats = result_list[0]
    print("=== Win Rate of {}, {} ===".format(name, stats["Win Rate [%]"]))

=== Win Rate of EURUSD, 37.5 ===
=== Win Rate of GBPUSD, 44.26229508196721 ===
=== Win Rate of USDCHF, 39.682539682539684 ===
=== Win Rate of USDJPY, 43.47826086956522 ===
